# 使用google Colab执行此代码

https://colab.research.google.com/drive/1G9SOcUK2x1Yl4dMdcY8i3fjmzdO-7Hzc#scrollTo=BkUs0rOfnJ07

## 查看虚拟机信息

In [1]:
print("============查看GPU信息================")
# 查看GPU信息
!nvidia-smi
print("==============查看pytorch版本==============")
# 查看pytorch版本
import torch
print(torch.__version__)
print("============查看虚拟机硬盘容量================")
# 查看虚拟机硬盘容量
!df -lh
print("============查看cpu配置================")
# 查看cpu配置
!cat /proc/cpuinfo | grep model\ name
print("=============查看内存容量===============")
# 查看内存容量
!cat /proc/meminfo | grep MemTotal


============查看GPU信息================
Thu Jul 11 11:35:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.40                 Driver Version: 536.40       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  | 00000000:01:00.0  On |                  N/A |
| 40%   46C    P0              27W /  90W |   1362MiB /  4096MiB |      1%      Default |
|                                         |                      |                  N/A |
+-------------------------------

'df' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


=============查看内存容量===============


'cat' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'cat' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


## 查看torch信息

In [2]:
import torch
print(torch.__version__)  #注意是双下划线

2.0.1+cu117


## 下载git

In [ ]:
!apt-get install git -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


## github下载minmal-minmal

In [ ]:
!git clone https://github.com/johnma2006/mamba-minimal.git

Cloning into 'mamba-minimal'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 57 (delta 29), reused 23 (delta 23), pack-reused 24
Receiving objects: 100% (57/57), 19.82 KiB | 2.83 MiB/s, done.
Resolving deltas: 100% (29/29), done.


## 测试

In [ ]:
cd /content/mamba-minimal

/content/mamba-minimal


In [ ]:
pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
from model import Mamba, ModelArgs
from transformers import AutoTokenizer

# One of:
#     'state-spaces/mamba-2.8b-slimpj'
#     'state-spaces/mamba-2.8b'
#     'state-spaces/mamba-1.4b'
#     'state-spaces/mamba-790m'
#     'state-spaces/mamba-370m'
#     'state-spaces/mamba-130m'
pretrained_model_name = 'state-spaces/mamba-370m'

model = Mamba.from_pretrained(pretrained_model_name)
tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neox-20b')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.49G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/457k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import torch
import torch.nn.functional as F


def generate(model,
             tokenizer,
             prompt: str,
             n_tokens_to_gen: int = 50,
             sample: bool = True,
             top_k: int = 40):
    model.eval()

    input_ids = tokenizer(prompt, return_tensors='pt').input_ids

    for token_n in range(n_tokens_to_gen):
        with torch.no_grad():
            indices_to_input = input_ids
            next_token_logits = model(indices_to_input)[:, -1]

        probs = F.softmax(next_token_logits, dim=-1)
        (batch, vocab_size) = probs.shape

        if top_k is not None:
            (values, indices) = torch.topk(probs, k=top_k)
            probs[probs < values[:, -1, None]] = 0
            probs = probs / probs.sum(axis=1, keepdims=True)

        if sample:
            next_indices = torch.multinomial(probs, num_samples=1)
        else:
            next_indices = torch.argmax(probs, dim=-1)[:, None]

        input_ids = torch.cat([input_ids, next_indices], dim=1)

    output_completions = [tokenizer.decode(output.tolist()) for output in input_ids][0]

    return output_completions

In [ ]:
print(generate(model, tokenizer, 'Mamba is the'))

Mamba is the latest project on which they are creating. The two-screen game takes players on a ride through the world of the Mamba universe before ending in an exploding planet on the planet of Mars. It’s a combination of sci-fi and urban


In [ ]:
print(generate(model, tokenizer, 'John: Hi!\nSally:'))

John: Hi!
Sally: Hi John! How're you?
John: Good Good good good!
Sally: You're great! Hows the health?
John: I'm good, how are you?
Sally: I'm very good, all better


In [ ]:
print(generate(model, tokenizer, 'The meaning of life is '))

The meaning of life is 獒未死. The expression comes from a poem by Tsugaru Shigenobu, first published in 1859.

See also
 Japanese poetry

References

Category:Japanese poetry
Category:Japanese language


In [ ]:
print(generate(model, tokenizer, 'def reverse_string('))

def reverse_string(self, string):
            if self._rnd_count is not None:
                return random.randint(self._rnd_count, self._size) + 1
            return self._key_counter.pop()



